# Edward and Mack Final Assignment Machine Learning 445 

In [20]:
%pip install bayesian-optimization torch torchvision pandas seaborn matplotlib scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from bayes_opt import BayesianOptimization as bo
import pandas as pd
import seaborn as sns
import time
from sklearn.metrics import confusion_matrix
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import psutil
import os

device = torch.device('mps' if torch.mps.is_available() else 'cpu')
print("Using device:", device)

#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


Using device: mps


In [3]:
pip install "datasets[vision]" pillow  # CPU-only wheel; add 'torch' separately


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 7.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
Note: you may need to restart the kernel to use updated packages.


In [25]:
from datasets import load_dataset
import sklearn 
from sklearn.model_selection import train_test_split
import torchvision.transforms as T

from datasets import load_dataset

ds = load_dataset("zh-plus/tiny-imagenet")
test_set  = ds["valid"]                      # final test set (10 k)

# Split the 100 k training images into 90 k train / 10 k val
splits = ds["train"].train_test_split(test_size=0.10, seed=42, stratify_by_column="label")

train_set  = splits["train"]
val_set    = splits["test"]

print(len(train_set), len(val_set), len(test_set))  # 90000 10000 10000


normalize = T.Normalize(mean=0.485, 0.456, 0.406),
                        std = (0.229, 0.224, 0.225)

train_tfms = T.Compose([
    T.RandomResizedCrop(64, scale=(0.8, 1.0), ratio=(0.9, 1.1)), 
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406),
                std =(0.229, 0.224, 0.225)),
])

eval_tfms = T.Compose([
    T.Resize(64),          
    T.CenterCrop(64),      
    T.ToTensor(),
    T.Normalize(mean=(0.485, 0.456, 0.406),
                std =(0.229, 0.224, 0.225)),
])

90000 10000 10000


# DataLoaders